## Affine&Softmax层的实现

### Affine层

这里的affine层其实就是说神经网络中的，信号加权的计算过程，也就是XW+B这个公式的计算。

> Affine表示仿射的意思，这是几何的概念，仿射一般是指对图形的平移或者线性伸缩。也就对应这里的对于输入信号的权重的线性变换和偏置的平移变换。

如下coding

In [4]:
import numpy as np

# 输入
X = np.random.rand(2)
# 权重 W
W = np.random.rand(2,3)
# 偏置值 B
B = np.random.rand(3)

print(X.shape,X)
print(W.shape,W)
print(B.shape,B)

Y = np.dot(X,W) + B
print(Y.shape,Y)

(2,) [0.03522538 0.33746387]
(2, 3) [[0.71601237 0.59287577 0.8510641 ]
 [0.57232106 0.74278995 0.62142448]]
(3,) [0.68368357 0.8806054  0.13857277]
(3,) [0.90204306 1.15215445 0.37826014]


从上面的例子中我们可以看到的数据是,以上的数据都是向量或者矩阵。Y是经过矩阵的点乘获得的，其矩阵变换如下：
![](imgs/19.jpg)

计算图如下：
![](imgs/20.jpg)

上面所有的数据都是多维数组的格式。

反向传播的过程中，对于向量的点乘t运算，其实这个也就是相当于标量的乘法求导，这里要注意的是：
- 向量求导要注意两个数据的位置
- 其次是要注意需要转置
计算图如下：
![](imgs/21.jpg)

这里有一个推导的过程如下：
![](imgs/22.jpg)
